# Controllable loop process
> iteration with callbacks

In [1]:
# default loop

In [2]:
# export 
import numpy as np

In [22]:
# export
from tqdm import tqdm
class StorageCore:
    def __init__(self,start_layer):
        self.layers = []
        self.lmap = dict()
        self.new_layer(start_layer)
        self.i = 0
    
    def new_layer(self,layer):
        layer.core = self
        self.layers.append(layer)
        self.lmap[layer._loop_layer]=layer
        setattr(self,layer.name,layer)
        
    def __repr__(self):
        return str(self.lmap)

class Loop:
    _loop_layer = 0
    def __init__(self,iterable,name=None):
        self.iterable  = iterable
        self.name = name if name!=None else self.__class__.__name__
        
        if hasattr(iterable,"_loop_layer"):
            self._loop_layer = iterable._loop_layer + 1
            iterable.core.new_layer(self)
        else:
            self.core = StorageCore(self)
            
        self.i = 0
        
    def __call__(self):
        pass
    
    def __repr__(self):
        return f"layer:{self.name}"
    
    def __len__(self):
        return self.iterable.__len__()
    
    def run(self,):
        for element in self:
            pass
            
    def __iter__(self,):
        if self._loop_layer == 0:
            for i,element in enumerate(self.iterable):
                self.core.i = i
                self.core.element = element
                self()
                yield self.core.element
        else:
            for element in self.iterable:
                self.core.element = element
                self()
                yield element

In [23]:
l = Loop(range(5,10),"hahahha")

In [29]:
from datetime import datetime

In [48]:
class Printer(Loop):
    def __call__(self):
        print(self.core.i,end="\t")
        print(self.core.element)

class ProgressBar(Loop):
    def __init__(self,iterable,jupyter = True):
        super().__init__(iterable,"Progress")
        if jupyter:
            from tqdm import tqdm_notebook as tq
        else:
            from tqdm import tqdm as tq
        self.tq = tq
        self.data = dict()
        
    def __call__(self):
        self.t.set_postfix(self.data)
        
    def set_data(self,dict_):
        self.data = dict_
        
    def __iter__(self):
        i = self.core.i
        if i==0:
            self.t = self.tq(self.iterable)
            self.gen = iter(self.t)
        
        for element in self.t:
            self()
            yield element
            
class Application(Loop):
    def __call__(self):
        self.core.Progress.set_data(dict(val = self.core.element,time = datetime.now()))

In [49]:
l2 = Printer(l)
l2.run()

0	5
1	6
2	7
3	8
4	9


In [50]:
l3 = Application(ProgressBar(Loop(range(5,10))))
l3.run()

In [227]:
l2.core

{0: layer:hahahha, 1: layer:ProgressBar}

In [183]:
l.run()

In [127]:
# export
class CallBack:
    """
    Use as ancestor
    class Printer(CallBack):
        def __call__(self,loop):
            print(loop.i,end="\t")
            print(loop.element)
    """
    def __init__(self,name=None):
        
        self.status = True
        if hasattr(self,"__call__")==False:
            raise NotImplementedError(f"Please define __call__(self,loop) function of callback: {self.name}")
    
    
    
    @property
    def i(self):
        return self.loop.i

In [20]:
l = Loop(range(10,20),)
l.run()